### Imports & Settings

In [15]:
# Load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dummy baseline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Load Data

In [9]:
# Cargar csv en un DataFrame
df = pd.read_csv("cars_data.csv")

# Explorar la estructura del DataFrame
df.info()
df.shape
df.isna().sum().sort_values(ascending=False)

# Ver el balance de las clases (84% Media, 10.5% Alta, 5.5% Baja)
df["calidad_auto"].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    10000 non-null  object 
 1   year                    10000 non-null  int64  
 2   selling_price           10000 non-null  int64  
 3   km_driven               10000 non-null  int64  
 4   fuel                    10000 non-null  int64  
 5   combustible_estimado_l  10000 non-null  float64
 6   seller_type             10000 non-null  int64  
 7   transmission            10000 non-null  int64  
 8   owner                   10000 non-null  int64  
 9   tipo_carroceria         10000 non-null  int64  
 10  potencia_motor_hp       10000 non-null  int64  
 11  nivel_seguridad         10000 non-null  float64
 12  calidad_auto            10000 non-null  object 
 13  score_calidad           10000 non-null  float64
 14  eficiencia_km_l         10000 non-null 

,proportion
calidad_auto,
Media,0.8399
Alta,0.1052
Baja,0.0549


#### Lo que esto nos dice
- Un fuerte desbalanceo de clases
  - "Media" es la mayoria con un 84%
  - "Alta" solo cubre el 10%, y "Baja" solo un 5.5%
- El accuracy de referencia (baseline) es engañosamente alto
  - Un DummyClassifier que siempre predice "Media" deberia de poder predecir 83.99%.
  - Superar este baseline no garantiza que el modelo sea mas precizo detectando "Alta" o "Baja".
- Riesgo de abandono de las clases minoritarias
  - El modelo tiene el riesgo de predecir "Media" la mayoria del tiempo debido al desbalanceo de las clases, lo que podria ignorar "Alta" o "Baja" como opciones.

### Exploratory Data Analysis & Baseline
#### Establecer una Dummy Baseline
- El primer paso es establecer una baseline y asegurarse de que la informacion esta limpia
- El objetivo es confirmar ques este DummyClassifier de como resultado una precision de 83.99%
- Se usara la estrategia de "most_frequent", simplemente el valor que mas se repite sera elegido para la prediccion.
- Este modelo representa el piso (baseline) de qualquier modelo futuro.



In [18]:
# Establecer la Dummy Baseline
X = df.drop("calidad_auto", axis=1)
y = df["calidad_auto"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Iniciar DummyClassifier
dummy = DummyClassifier(
    strategy="most_frequent", # Siempre predice la clase mas frecuente ("Media")
    random_state=42
)
dummy.fit(X_train, y_train)   # "Entrena" basado en la clase mas frecuente

# Obtener predicciones
y_pred = dummy.predict(X_test)  # Aplica la regla a cada fila de X_test

# Evaluar accuracy
accuracy = accuracy_score(y_test, y_pred)   # Fraccion de predicciones correctas
print(f"DummyClassifier Accuracy: {accuracy}")

# Visualizar resulatdos de predicciones
print("Classification report:\n")
print(classification_report(y_test, y_pred))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

DummyClassifier Accuracy: 0.84
Classification report:

              precision    recall  f1-score   support

        Alta       0.00      0.00      0.00       210
        Baja       0.00      0.00      0.00       110
       Media       0.84      1.00      0.91      1680

    accuracy                           0.84      2000
   macro avg       0.28      0.33      0.30      2000
weighted avg       0.71      0.84      0.77      2000

Confusion Matrix:
 [[1680    0    0]
 [ 210    0    0]
 [ 110    0    0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Lo que esto nos dice
- Tenemos un baseline de ~84% accuracy, este es el objetivo a vencer en los proximos modelos.

### Features

### Modeling

### Evaluation